##  Inception

여기서 구현하는 모델은 다음과 같다.

![inception.png](images/inception.png)

이 모델(정확한 세부 구현은 달라질 수 있다)을 모듈로 여러 개 이어 붙여서 Inception Network를 구성할 수 있다.

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

#CNN에서 결국 필터의 수만큼의 output이 나오게 된다.
#pooling을 하면, 채널(필터 수)는 그대로 이고 사이즈만 줄어든다.
#Linear로도 이미지를 판별할 수 있다. 하지만 CNN이 정확도가 훨씬 높다. 
#Linear로 연결하면, 모든 가중치가 한 번에 계산되고 업데이트 되지만, CNN은 작은 단위 별로 각 가중치가 계산되고 업데이트 되면서 더 유연하게 판별할 수 있다.

In [2]:
# Training settings
batch_size = 64

#1. epoch : 모든 데이터를 한 번씩 forward, backward 한 상태
#2. batch_size : 해당 크기 만큼의 데이터를 forward, backward 한다. 높을 수록 전체 학습속도는 빨라지지만, 메모리가 많이 필요하다.
#3. iterations : 몇 번의 forward, backward를 진행 했는지. batch_size 만큼 forward, backward 한다.

#1000개의 data가 있을 때 batch_size를 500으로 하면, 2 번의 iterations이 지났을 때 1 epoch이 완료된다.

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())
#transforms은 선처리를 위한 함수. transforms.Compose로 컨테이너에 묶어 줄 수 있다.

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
#데이터를 training 용과 test 용으로 나눠야 한다.
#이미 선언되어 있는 데이터 셋을 가져온다. .을 붙여줘야 된다(안 붙이면 pemission denied).
#Dataset을 DataLoader로 불러와 쉽게 batch를 만들 수 있다.

In [3]:
class InceptionA(nn.Module): #Module상속 
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        
        #inception model은 여러 필터(1 x 1, 3 x 3 ...)의 결과를 합쳐서(Concatenate) 학습한다.  
        #Conv2d : (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
        #pooling을 지나면, 채널은 그대로이고 사이즈가 준다. 필터의 수 만큼 반복되므로, 필터의 수가 out_channels 이 된다.
        #채널은 필터의 수라 생각하면 된다.
        
        #합칠 때의 size는 동일해야 하므로, 필터를 적용할 때 padding을 적당히 줘서 결과의 size를 같도록 한다(channel, 필터의 수는 상관 x).    
        #Inception에서는 1 x 1 필터를 사용하는 경우가 많은데, 사실 1 x 1의 결과물 자체는 큰 의미가 없지만, channel을 조정할 수 있다.    
        #channel이 줄어들 수록, 변수가 적어지므로 연산 시 부하를 줄일 수 있다. 또한, 의미 있는 필터가 조합되므로 성능이 향상되기도 한다. 
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1) #average pooling
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool] 
        return torch.cat(outputs, 1) #여러 필터들의 결과를 합쳐준다. 1은 dimension
        #dim을 0으로 주면 각 행렬이 밑으로 붙고, 1로 주면 오른쪽으로 붙는다.

In [4]:
class Net(nn.Module): #Module상속 
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)
        #Conv2d : (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
        #pooling을 지나면, 채널은 그대로이고 사이즈가 준다. 필터의 수 만큼 반복되므로, 필터의 수가 out_channels 이 된다.
        #채널은 필터의 수라 생각하면 된다.

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)
        #Inception을 모듈로 넣어 줄 수 있다.
        #Inception이 많아진다고 항상 정확도가 향상하는 것은 아니다(Vanishing gradients). 
        #이를 해결하기 위해 나온 모델이 Deep Residual Neural Network
        
        self.mp = nn.MaxPool2d(2)
        #MaxPool2d : (kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        #Max pooling에 stride를 따로 주지 않으면, 기본적으로 kernel_size에 맞춘다. 따라서 여기서는 stride = 2가 된다.
        #여기서는 2 * 2 크기로 2칸 씩 이동하므로 절반으로 줄어든다.
        
        self.fc = nn.Linear(1408, 10)
        #(in_features, out_features, bias=True)
        #FC를 만들 때 input features를 계산하기 힘들다면, 그냥 아무 숫자나 넣고 running한다. 
        #그러면 error message에서 차원을 확인할 수 있다. #ex. RuntimeError: size mismatch, m1:[64 x 320], m2: [100 x 10] 
        #64 x 320에서 64는 batch_size이고, 320가 input이 될 features가 된다.

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x) #마지막엔 FC로 연결해 준다.
        return F.log_softmax(x, dim=1) #활성함수는 log softmax

model = Net() #model 생성

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) #최적화함수
#model.parameters() 로 업데이트해야 할 모든 변수들을 한 번에 가져와 간단히 구현할 수 있다.
#손실에서 lr만큼 움직이고 업데이트 하던 것을 알아서 최적화해서 처리해 준다.

In [6]:
def train(epoch):
    model.train() #train 임을 알려준다.
    for batch_idx, (data, target) in enumerate(train_loader): #DataLoader를 사용하면 batch만큼 가져온다.
        optimizer.zero_grad() #optimiser.step() 으로 업데이트된 그라디언트 값들을 초기화해 줘야 한다.
        output = model(data) #예측 값 #4.0부터는 Variable 없이 그냥 Tensor를 그대로 넣어도 된다.
        loss = F.nll_loss(output, target) #손실함수에 예측한 값과, 정답을 넣어 loss를 구한다.
        loss.backward() #역전파 해 준다. 각 변수의 기울기를 구한다.
        optimizer.step() #변수 업데이트
        if batch_idx % 10 == 0: #10번째 마다 출력
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())) #4.0 부터는 data[0] 대신 item()을 사용하도록 바꼈다.


def test():
    model.eval() #eval() 임을 알려준다. eval()을 써야 변수가 고정되어 제대로 평가할 수 있다.
    test_loss = 0
    correct = 0
    for data, target in test_loader: #DataLoader를 사용하면 batch만큼 가져온다.
        output = model(data) #예측 값 #4.0부터는 Variable 없이 그냥 Tensor를 그대로 넣어도 된다.
        # sum up batch loss
#         test_loss += F.nll_loss(output, target, size_average=False).data[0]
        test_loss += F.nll_loss(output, target, reduction='sum').item() #손실함수에 예측한 값과, 정답을 넣어 loss를 구한다.
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1] #예측 값에서 확률이 가장 높은 것을 가져온다.
        correct += pred.eq(target.data.view_as(pred)).cpu().sum() #정확도를 구한다.
        #pred.eq을 하면 pred와 data가 일치하는 지 검사한다. 
        #view_as(pred)로, 같은 shape로 변환한후,
        #.sum()으로 일치하는 요소들의 수를 더해 가져온다.

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
#train과 test를 클래스로 만들어 준다.

In [7]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296074
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.295360
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297848
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.302992
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.301280
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.294357
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.292838
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.300146
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.299965
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.287901
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.290064
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.301051
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.286815
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.291850
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.298246
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.286202
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.281807
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.292177
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.273755
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.248767
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.141254
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.088310
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.038942
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.060920
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.157575
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.057126
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.081634
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.139310
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.114071
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.081900
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.042254
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.118027
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.174607
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.134618
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.106733
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.166047
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.068315
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.061907
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.101836
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.253137


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.058397
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.054235
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.038668
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.123567
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.206309
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.138457
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.055712
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.061273
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.124467
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.048463
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.016804
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.064237
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.022298
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.012589
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.063456
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.499480
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.028929
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.256776
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.107082
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.098075


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.044736
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.054805
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.081328
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.023595
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.037419
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.094804
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.009000
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.060578
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.066145
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.021390
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.120223
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.074175
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.052542
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.162591
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.063550
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.120712
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.086271
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.129635
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.071498
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.108384


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.015462
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.010519
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.014114
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.065952

Test set: Average loss: 0.0491, Accuracy: 9845/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.018061
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.068653
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.020614
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.095518
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.059489
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.227932
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.030238
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.016041
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.009287
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.070655
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.056891
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.092257
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.089131
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.104842
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.080696
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.015063
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.050231
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.030358
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.047714
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.097623
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.074890
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.014580
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.091360
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.021455
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.005700
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.045084
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.221430
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.033339
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.015619
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.065752
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.038501
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.105683
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.085827
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.120294
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.013339
